Create a program that asks the user for their age, calculates how many years until they turn 100, and prints the result.

In [ ]:
age = int(input("Enter your current age : "))
if age > 100:
    print("You are more than 100 years old")
elif age == 100:
    print("You are exactly 100 years old")
else:
    print(" You will turn 100 years old in ",(100 - age))

Enter your current age : 1465
You are more than 100 years old


Write a program that asks for a number and tells whether it is positive, negative, or zero.

In [ ]:
num = float(input("Enter a number : "))
if num > 0:
    print("The number (",num,") is positive")
elif num < 0:
    print("The number (",num,") is negative")
else:
    print("The number (",num,") is zero")

Enter a number : 0.000000
The number ( 0.0 ) is zero


Write a Python program that accepts a list of integers from the user and prints out only the even numbers using a for loop. Also, print the sum of those even numbers.

In [ ]:
N = int(input("Enter how many numbers you want to input : "))
num = []
even = []
sum = 0
for i in range(N):
    inp_num = int(input("Enter the number : "))
    num.append(inp_num)
    if inp_num % 2 == 0:
        even.append(inp_num)
        sum += inp_num
    else:
        continue
print("The numbers you gave are : ",num)
print("The even numbers among them are : ",even)
print("The sum of even numbers is : ",sum)


Enter how many numbers you want to input : 3
Enter the number : 66
Enter the number : 2
Enter the number : 11
The numbers you gave are :  [66, 2, 11]
The even numbers among them are :  [66, 2]
The sum of even numbers is :  68


Write a program to take inputs in a dictionary and print the dictionary


In [ ]:
dictionary = {}
num_pairs = int(input("Enter the number of key-value pairs: "))
for i in range(num_pairs):
    key = input(f"Enter key {i+1}: ")
    value = input(f"Enter value for key '{key}': ")
    dictionary[key] = value
print("The dictionary is:", dictionary)

Enter the number of key-value pairs: 3
Enter key 1: name
Enter value for key 'name': Aritra
Enter key 2: age
Enter value for key 'age': 20
Enter key 3: subject
Enter value for key 'subject': Physics
The dictionary is: {'name': 'Aritra', 'age': '20', 'subject': 'Physics'}


To fix this, you need to create a separate file called mymodule.py with the function definition and then import it.

Here is the breakdown of steps:

Create a file named mymodule.py in the same directory as your notebook.
Move the function definition into mymodule.py.
Import the module using import mymodule.

# mymodule.py

def yourbirthyear(age, year):
    """Calculates and prints the birth year."""  # Added a docstring for clarity
    print("Your Birth year is : ", (year - age))
Use code with caution

# Your main script or notebook cell

import mymodule  # Imports the mymodule.py file

mymodule.yourbirthyear(21, 2025)  # Calls the function from the module
Use code with caution
Explanation of changes:

We moved the yourbirthyear function definition to a separate file named mymodule.py. This creates a proper Python module that can be imported.
We then imported this module using import mymodule in our main script or notebook cell.
Finally, we called the function using mymodule.yourbirthyear(21, 2025). This executes the code within the function defined in mymodule.py.

In [ ]:
# mymodule.py
def yourbirthyear(age,year):
    print ("Your Birth year is : ", (year-age))

import mymodule
mymodule.yourbirthyear(21,2025)

ModuleNotFoundError: No module named 'mymodule'

ModuleNotFoundError: No module named 'mymodule'

In [ ]:
# prompt: Write a python function with positional arguments, default arguments

def Pythagoras(a, b, a_pow=2, b_pow=2):
    return (a**a_pow + b**b_pow)**0.5
a = float(input("Enter the value of a : "))
b = float(input("Enter the value of b : "))
print("Value of c is : ",Pythagoras(a, b))

Enter the value of a : 4
Enter the value of b : 1
Value of c is :  4.123105625617661


In [1]:

import math
import uuid
from flask import Flask, render_template, request, redirect, url_for, session
import plotly.graph_objects as go
import numpy as np
import re

app = Flask(__name__)
app.secret_key = 'your_secret_key'  # Used for session management

# Store graphs using a dictionary (Graph ID -> Equations List)
graph_store = {}

# Map simplified functions to math module functions
math_functions = {
    'sin': 'math.sin',
    'cos': 'math.cos',
    'tan': 'math.tan',
    'log': 'math.log',
    'sqrt': 'math.sqrt',
    'exp': 'math.exp',
    'abs': 'abs',  # No need to map, Python already has abs()
}

def generate_plot(equations):
    """Generate a Plotly graph for the given equations."""
    x_vals = np.linspace(-10, 10, 400)
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'pink']
    traces = []

    for i, expression in enumerate(equations):
        # Replace simplified function names with math module equivalents
        for func, replacement in math_functions.items():
            expression = re.sub(rf'\b{func}\b', replacement, expression)

        y_vals = []
        try:
            y_vals = [eval(expression.replace('x', f'({x})')) for x in x_vals]
        except Exception as e:
            return f"Error in expression: {e}"

        traces.append(go.Scatter(x=x_vals, y=y_vals, mode="lines", name=expression, line=dict(color=colors[i % len(colors)])))

    fig = go.Figure(data=traces)
    fig.update_layout(title="Multiple Equations",
                      xaxis_title="x", yaxis_title="y",
                      legend_title="Equations")

    return fig.to_html(full_html=False)

@app.route("/", methods=["GET", "POST"])
def index():
    if request.method == "POST":
        action = request.form.get("action")

        if action == "add":
            expression = request.form["expression"]
            graph_id = session.get("graph_id")

            # If no graph ID exists, generate a new one
            if not graph_id:
                graph_id = str(uuid.uuid4())  # Generate unique ID
                session["graph_id"] = graph_id  # Save in session
                graph_store[graph_id] = []  # Initialize graph storage

            graph_store[graph_id].append(expression)  # Store equation

            return redirect(url_for("graph", graph_id=graph_id))

        elif action == "new":
            session.pop("graph_id", None)  # Clear session graph ID
            return redirect(url_for("index"))

    return render_template("index.html", graph_html="", graph_id=None, equations=[])

@app.route("/graph/<graph_id>", methods=["GET", "POST"])
def graph(graph_id):
    equations = graph_store.get(graph_id, [])

    if request.method == "POST":
        action = request.form.get("action")

        if action == "remove":
            equation_index = int(request.form["equation_index"])
            equations.pop(equation_index)
            graph_store[graph_id] = equations  # Update stored equations

        return redirect(url_for("graph", graph_id=graph_id))

    graph_html = generate_plot(equations) if equations else ""

    return render_template("index.html", graph_html=graph_html, graph_id=graph_id, equations=equations)

if __name__ == "__main__":
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [3]:

!pip install flask flask-ngrok numpy sympy plotly pyngrok cloudflared

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.5 MB/s eta 0:00:00
  Created wheel for cloudflared: filename=cloudflared-1.0.0.2-py3-none-any.whl size=2984 sha256=71bdcfdce57d0eec5221879d0c74eb526422809e6085a2538c832cf5a7146342
  Stored in directory: /root/.cache/pip/wheels/3c/9f/f1/ef5e36c9386d737ac05ab8714d611c430d79abe55d862ca2b7
Successfully built cloudflared


In [1]:
!pip install flask
!pip install pyngrok

In [6]:
import math
import uuid
from flask import Flask, render_template, request, redirect, url_for, session
import plotly.graph_objects as go
import numpy as np
import re
from pyngrok import ngrok, conf

app = Flask(__name__)
app.secret_key = 'your_secret_key'  # Used for session management

# Store graphs using a dictionary (Graph ID -> Equations List)
graph_store = {}

# Map simplified functions to math module functions
math_functions = {
    'sin': 'math.sin',
    'cos': 'math.cos',
    'tan': 'math.tan',
    'log': 'math.log',
    'sqrt': 'math.sqrt',
    'exp': 'math.exp',
    'abs': 'abs',  # No need to map, Python already has abs()
}

def generate_plot(equations):
    """Generate a Plotly graph for the given equations."""
    x_vals = np.linspace(-10, 10, 400)
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'pink']
    traces = []

    for i, expression in enumerate(equations):
        # Replace simplified function names with math module equivalents
        for func, replacement in math_functions.items():
            expression = re.sub(rf'\b{func}\b', replacement, expression)

        y_vals = []
        try:
            y_vals = [eval(expression.replace('x', f'({x})')) for x in x_vals]
        except Exception as e:
            return f"Error in expression: {e}"

        traces.append(go.Scatter(x=x_vals, y=y_vals, mode="lines", name=expression, line=dict(color=colors[i % len(colors)])))

    fig = go.Figure(data=traces)
    fig.update_layout(title="Multiple Equations",
                      xaxis_title="x", yaxis_title="y",
                      legend_title="Equations")

    return fig.to_html(full_html=False)

@app.route("/", methods=["GET", "POST"])
def index():
    if request.method == "POST":
        action = request.form.get("action")

        if action == "add":
            expression = request.form["expression"]
            graph_id = session.get("graph_id")

            # If no graph ID exists, generate a new one
            if not graph_id:
                graph_id = str(uuid.uuid4())  # Generate unique ID
                session["graph_id"] = graph_id  # Save in session
                graph_store[graph_id] = []  # Initialize graph storage

            graph_store[graph_id].append(expression)  # Store equation

            return redirect(url_for("graph", graph_id=graph_id))

        elif action == "new":
            session.pop("graph_id", None)  # Clear session graph ID
            return redirect(url_for("index"))

    return render_template("index.html", graph_html="", graph_id=None, equations=[])

@app.route("/graph/<graph_id>", methods=["GET", "POST"])
def graph(graph_id):
    equations = graph_store.get(graph_id, [])

    if request.method == "POST":
        action = request.form.get("action")

        if action == "remove":
            equation_index = int(request.form["equation_index"])
            equations.pop(equation_index)
            graph_store[graph_id] = equations  # Update stored equations

        return redirect(url_for("graph", graph_id=graph_id))

    graph_html = generate_plot(equations) if equations else ""

    return render_template("index.html", graph_html=graph_html, graph_id=graph_id, equations=equations)

# Start the Flask app
if __name__ == "__main__":
    # 1. Get your authtoken from https://dashboard.ngrok.com/auth
    # 2. Set your authtoken in the ngrok configuration
    conf.get_default().auth_token = "2tlT630VkyCzjn4AbFnNkC056V8_6HfhAcf4ZPyoMNETXHfmH"  # Replace with your authtoken

    # Set up ngrok
    public_url = ngrok.connect(5000)
    print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:5000\"")
    app.run()

 * ngrok tunnel "NgrokTunnel: "https://1bb6-34-28-100-145.ngrok-free.app" -> "http://localhost:5000"" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
ERROR:__main__:Exception on / [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
          


Day 11:

1. Download gitbash on windows.
2. Git helps in version control.
3. Working area->Staging area->Git repository--》 Remote repository
4. MAKE a readme.md markdown

In [ ]:
#file-1.txt
# It was the best of times, it was the worst of times

!git add file-1.txt # using add . Adds everything to staging area
!git commit -m"initial commit" # -m for message
!git status
!git log
!git branch
!git push branch
!git checkout -b dev
!git push origin -u <new branch name>

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 52.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of pymc3 to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 65.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.5/869.5 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 10.2 MB/s eta 0:00:00
  Created wheel for theano-pymc: filename=Theano_PyMC-1.1.2-py3-none-any.whl size=1529959 sha256=f73377a556b63822ca9ddc71230c0fd2daf23cd2deac19c200a73475dcdce820
  Stored in directory: /root/.cache/pip/wheels/fb/d3/25/6c443075ee21f3b6a59a6a180b55657b012722e1b89f8ccbe8
Successfully built theano-pymc


In [ ]:
!pip install --upgrade theano
!pip install --upgrade pymc3
import pymc3 as pm
import numpy as np

# Supernova data (example, replace with real data)
z_data = np.array([0.1, 0.2, 0.5, 1.0])  # Redshifts
d_L_obs = np.array([400, 800, 2500, 6000])  # Luminosity distances in Mpc
d_L_err = np.array([50, 100, 200, 300])  # Errors in distances

# Define the theoretical luminosity distance function
def luminosity_distance(H0, Omega_m, z):
    Omega_Lambda = 1 - Omega_m  # Assume a flat universe
    integral = np.array([np.trapz(1 / np.sqrt(Omega_m * (1 + z_prime) ** 3 + Omega_Lambda), z_prime)
                         for z_prime in z])
    return (299792.458 / H0) * (1 + z) * integral  # c/H0 factor

# Bayesian modeling with PyMC3
with pm.Model() as model:
    H0 = pm.Normal("H0", mu=70, sigma=10)  # Prior for Hubble constant
    Omega_m = pm.Uniform("Omega_m", 0, 1)  # Prior for matter density
    d_L_pred = luminosity_distance(H0, Omega_m, z_data)
    d_L_obs_likelihood = pm.Normal("d_L_obs", mu=d_L_pred, sigma=d_L_err, observed=d_L_obs)
    trace = pm.sample(1000, tune=1000)

# Plot posterior distributions
import arviz as az
az.plot_posterior(trace, var_names=["H0", "Omega_m"])

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 74.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for theano: filename=Theano-1.0.5-py3-none-any.whl size=2668112 sha256=beb6aec8c3c006aa7374a8f24fc242d47f6736ecdb04ff5a9df9ab6dc2e1aa17
  Stored in directory: /root/.cache/pip/wheels/26/1f/2f/02d738022626461828148150c0354e712c4ad74f1a60f10933
Successfully built theano
  Using cached pymc3-3.11.6-py3-none-any.whl.metadata (15 kB)
  Using cached deprecat-2.1.3-py2.py3-none-any.whl.metadata (1.6 kB)
  Using cached numpy-1.22.1.zip (11.4 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of pymc3 to determine which version is compatible with other requirements. This could take a while.
  Using cached pymc3-3.11.5-py3-none-any.whl.metadata (14 kB)


ImportError: cannot import name 'local_bitwidth' from 'theano.configdefaults' (/usr/local/lib/python3.11/dist-packages/theano/configdefaults.py)